In [3]:
!python -m spacy download nl_core_news_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.1/12.8 MB 656.4 kB/s eta 0:00:20
      --------------------------------------- 0.2/12.8 MB 1.5 MB/s eta 0:00:09
     - -------------------------------------- 0.4/12.8 MB 2.4 MB/s eta 0:00:06
     - -------------------------------------- 0.5/12.8 MB 2.8 MB/s eta 0:00:05
     -- ------------------------------------- 0.7/12.8 MB 2.8 MB/s eta 0:00:05
     -- ------------------------------------- 0.8/12.8 MB 2.8 MB/s eta 0:00:05
     --- ------------------------------------ 1.1/12.8 MB 2.9 MB/s eta 0:00:05
     --- ------------------------------------ 1.1/12.8 MB 2.9 MB/s eta 0:00:05
     --- ------------------------------------ 1.2/12.8 MB 2.8 MB/s eta 0:00:05
     ---- ----------------------------------- 1.3/12.8 MB 2.6 MB/s eta 0:00:05
     ---- ----------------------------------- 1.4/12.8 MB 2.5 MB/

In [4]:
import spacy

In [5]:
nlp = spacy.load('nl_core_news_sm')

In [6]:
with open('C:/Users/Chloe/Documents/MaastrichtLaw&Tech/Summarization/test.txt') as f:
	texts = f.read().splitlines()

In [7]:
sents = []
for text in texts:
    doc = nlp(text)
    for sent in doc.sents:
        sents.append(sent)

In [8]:
MIN_LENGTH = 3
tokenized_sents = [[token.lemma_.lower() for token in sent 
                    if not token.is_stop and 
                    not token.is_punct and token.text.strip() and 
                    len(token) >= MIN_LENGTH] 
                  for sent in sents]

In [9]:
from gensim import corpora, models
N_TOPICS = 5
N_PASSES = 5
dictionary = corpora.Dictionary(tokenized_sents)
bow = [dictionary.doc2bow(sent) for sent in tokenized_sents]
topic_model = models.LdaModel(corpus=bow, id2word=dictionary, 
                              num_topics=N_TOPICS, passes=N_PASSES)

In [10]:
topic_model.show_topics()

[(0,
  '0.044*"overdracht" + 0.037*"verzoekster" + 0.034*"melding" + 0.032*"2018" + 0.030*"fosfaatrecht" + 0.028*"verweerder" + 0.018*"doen" + 0.017*"275" + 0.016*"blijken" + 0.016*"februari"'),
 (1,
  '0.064*"verzoekster" + 0.047*"fosfaatrecht" + 0.039*"2018" + 0.033*"verweerder" + 0.033*"besluit" + 0.025*"schade" + 0.023*"stellen" + 0.019*"januari" + 0.018*"september" + 0.011*"waarbij"'),
 (2,
  '0.026*"artikel" + 0.026*"lid" + 0.020*"besluit" + 0.018*"per" + 0.017*"awb" + 0.016*"verzoek" + 0.014*"onrechtmatig" + 0.013*"kg." + 0.013*"schade" + 0.012*"2018"'),
 (3,
  '0.043*"besluit" + 0.037*"verzoekster" + 0.037*"verweerder" + 0.029*"fosfaatrecht" + 0.028*"schade" + 0.019*"oktober" + 0.019*"standpunt" + 0.014*"vaststellen" + 0.014*"stellen" + 0.014*"hoog"'),
 (4,
  '0.031*"verzoekster" + 0.025*"netto" + 0.024*"2019" + 0.024*"college" + 0.021*"verkopen" + 0.018*"fosfaatrecht" + 0.017*"besluit" + 0.015*"vast" + 0.014*"170,-" + 0.012*"januari"')]

In [11]:
THRESHOLD = 0.05
doc_topics = list(topic_model.get_document_topics(bow, minimum_probability=THRESHOLD))

In [12]:
print(doc_topics)

[[(0, 0.10002992), (1, 0.100017205), (2, 0.10002168), (3, 0.5999026), (4, 0.10002857)], [(2, 0.961691)], [(0, 0.100029826), (1, 0.100017145), (2, 0.100021616), (3, 0.59990275), (4, 0.10002863)], [(0, 0.21437459), (2, 0.7454135)], [(0, 0.100038074), (1, 0.10002187), (2, 0.59986943), (3, 0.10003419), (4, 0.10003638)], [(2, 0.9651155)], [(0, 0.10002993), (1, 0.10001716), (2, 0.10002176), (3, 0.5999026), (4, 0.10002858)], [(0, 0.10003821), (1, 0.10002198), (2, 0.59986943), (3, 0.10003408), (4, 0.10003636)], [(1, 0.919098)], [(0, 0.10002775), (1, 0.10001602), (2, 0.10002018), (3, 0.100024834), (4, 0.5999113)], [(0, 0.9327877)], [(0, 0.100029916), (1, 0.100017175), (2, 0.10002176), (3, 0.5999026), (4, 0.10002854)], [(1, 0.93277603)], [(1, 0.9329615)], [(3, 0.8835063)], [(0, 0.10002991), (1, 0.10001717), (2, 0.10002177), (3, 0.5999025), (4, 0.100028604)], [(0, 0.9180302)], [(0, 0.10002776), (1, 0.10001596), (2, 0.100020185), (3, 0.1000248), (4, 0.5999113)], [(0, 0.94627476)], [(0, 0.10002983)

In [13]:
k = 3
top_k_topics = [[t[0] for t in sorted(sent_topics, key=lambda x: x[1], reverse=True)][:k] 
                for sent_topics in doc_topics]

In [16]:
from itertools import islice
def window(seq, n=3):
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n: 
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [17]:
from itertools import chain
WINDOW_SIZE = 3
window_topics = window(top_k_topics, n=WINDOW_SIZE)
window_topics = [list(set(chain.from_iterable(window))) 
                for window in window_topics]

In [18]:
print(window_topics)

[[0, 2, 3, 4], [0, 2, 3, 4], [0, 2, 3, 4], [0, 2, 4], [0, 2, 3, 4], [0, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 3, 4], [0, 3, 4], [0, 1, 3, 4], [0, 1, 3, 4], [1, 3], [0, 1, 3, 4], [0, 3, 4], [0, 3, 4], [0, 3, 4], [0, 3, 4], [0, 1, 3, 4], [0, 1, 3, 4], [0, 1, 4], [0, 1, 3, 4], [0, 3, 4], [0, 1, 3, 4], [0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2], [0, 1, 2], [1], [1, 3], [1, 3, 4], [1, 3, 4], [1, 2, 3, 4], [1, 2, 3], [1, 2, 3], [1, 3], [0, 1, 3], [0, 1, 3, 4], [0, 1, 3, 4], [0, 3, 4], [0, 1, 3], [0, 1, 3], [0, 1, 3], [0, 1, 3], [0, 1, 2, 3], [1, 2, 3], [0, 1, 2, 3, 4], [0, 1, 3, 4], [0, 3, 4], [0, 2, 3], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1], [0, 1, 4], [1, 4], [1, 3, 4], [0, 1, 3, 4], [0, 3, 4], [0, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 2, 4], [0, 1, 2, 4], [0, 2, 3, 4], [0, 3, 4], [0, 3, 4], [1, 3, 4], [1, 3, 4], [1, 2, 4], [0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0, 2, 3, 4], [1, 2, 4], [0, 1, 2, 3, 4], [0, 1, 3, 4], [0, 2, 3, 4], [2, 3], [0, 2, 3, 4], [0, 1, 2, 3, 4], [0, 1, 3, 4], [0, 1

In [19]:
from sklearn.preprocessing import MultiLabelBinarizer
binarizer = MultiLabelBinarizer(classes=range(N_TOPICS))
encoded_topic = binarizer.fit_transform(window_topics)

In [20]:
print(encoded_topic)

[[1 0 1 1 1]
 [1 0 1 1 1]
 [1 0 1 1 1]
 [1 0 1 0 1]
 [1 0 1 1 1]
 [1 0 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 0 1 1]
 [1 0 0 1 1]
 [1 1 0 1 1]
 [1 1 0 1 1]
 [0 1 0 1 0]
 [1 1 0 1 1]
 [1 0 0 1 1]
 [1 0 0 1 1]
 [1 0 0 1 1]
 [1 0 0 1 1]
 [1 1 0 1 1]
 [1 1 0 1 1]
 [1 1 0 0 1]
 [1 1 0 1 1]
 [1 0 0 1 1]
 [1 1 0 1 1]
 [1 1 1 1 0]
 [1 1 1 1 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 1 0 0 0]
 [0 1 0 1 0]
 [0 1 0 1 1]
 [0 1 0 1 1]
 [0 1 1 1 1]
 [0 1 1 1 0]
 [0 1 1 1 0]
 [0 1 0 1 0]
 [1 1 0 1 0]
 [1 1 0 1 1]
 [1 1 0 1 1]
 [1 0 0 1 1]
 [1 1 0 1 0]
 [1 1 0 1 0]
 [1 1 0 1 0]
 [1 1 0 1 0]
 [1 1 1 1 0]
 [0 1 1 1 0]
 [1 1 1 1 1]
 [1 1 0 1 1]
 [1 0 0 1 1]
 [1 0 1 1 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [1 1 0 0 0]
 [1 1 0 0 1]
 [0 1 0 0 1]
 [0 1 0 1 1]
 [1 1 0 1 1]
 [1 0 0 1 1]
 [1 0 1 1 1]
 [1 1 1 1 1]
 [1 1 1 0 1]
 [1 1 1 0 1]
 [1 0 1 1 1]
 [1 0 0 1 1]
 [1 0 0 1 1]
 [0 1 0 1 1]
 [0 1 0 1 1]
 [0 1 1 0 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 0 1 1 1]
 [0 1 1 0 1]
 [1 1 1 1 1]
 [1 1 0 1 1]
 [1 0 1 1 1]
 [0 0 1 1 0]

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
coherence_scores = [cosine_similarity([pair[0]], [pair[1]])[0][0] 
                for pair in zip(encoded_topic, encoded_topic[1:])]

In [22]:
def climb(seq, i, mode='left'):
    if mode == 'left':
        while True:
            curr = seq[i]
            if i == 0:
                return curr
            i = i-1
            if not seq[i] > curr:
                return curr
    if mode == 'right':
        while True:
            curr = seq[i]
            if i == (len(seq)-1):
                return curr
            i = i+1
            if not seq[i] > curr:
                return curr

In [24]:
import numpy as np
def get_depths(scores):
    depths = []
    for i in range(len(scores)):
        score = scores[i]
        l_peak = climb(scores, i, mode='left')
        r_peak = climb(scores, i, mode='right')
        depth = 0.5 * (l_peak + r_peak - (2*score))
        depths.append(depth)
    return np.array(depths)

In [28]:
depth_scores = get_depths(coherence_scores)

In [29]:
from scipy.signal import argrelmax

def get_local_maxima(depth_scores, order=1):
     maxima_ids = argrelmax(depth_scores, order=order)[0]
     filtered_scores = np.zeros(len(depth_scores))
     filtered_scores[maxima_ids] = depth_scores[maxima_ids]
     return filtered_scores

In [31]:
filtered_scores = get_local_maxima(depth_scores, order=1)

In [32]:
def compute_threshold(scores):
     s = scores[np.nonzero(scores)]
     threshold = np.mean(s) - (np.std(s) / 2)
     # threshold = np.mean(s) - (np.std(s))
     return threshold

In [33]:
threshold = compute_threshold(filtered_scores)

In [34]:
def get_threshold_segments(scores, threshold=0.1):
     segment_ids = np.where(scores >= threshold)[0]
     return segment_ids

In [35]:
segment_ids = get_threshold_segments(filtered_scores, threshold)

In [36]:
print(segment_ids)

[ 12  17  23  25  27  39  49  62  65  67  71  81  88  91  94  97  99 103]


In [37]:
segment_indices = segment_ids + WINDOW_SIZE

In [38]:
segment_indices = [0] + segment_indices.tolist() + [len(sents)]
slices = list(zip(segment_indices[:-1], segment_indices[1:]))

In [39]:
segmented = [sents[s[0]: s[1]] for s in slices]

In [40]:
print(segmented)

[[1.1, Op grond van artikel 8:88, eerste lid, aanhef en onder a, van de Awb is de bestuursrechter bevoegd op verzoek van een belanghebbende een bestuursorgaan te veroordelen tot vergoeding van schade die de belanghebbende lijdt of zal lijden als gevolg van een onrechtmatig besluit., 1.2, Artikel 8:90, eerste lid, van de Awb bepaalt dat het verzoek schriftelijk wordt ingediend bij de bestuursrechter die bevoegd is kennis te nemen van het beroep tegen het besluit., 1.3, Artikel 8:90, tweede lid, van de Awb bepaalt dat ten minste acht weken voor het indienen van het in het eerste lid bedoelde verzoekschrift de belanghebbende het betrokken bestuursorgaan schriftelijk om vergoeding van de schade vraagt, tenzij dit redelijkerwijs niet van hem kan worden gevergd., Feiten, 2.1, Bij besluit van 12 januari 2018 heeft verweerder het fosfaatrecht van verzoekster vastgesteld op 306 kg., 2.2, Verzoekster heeft op 8 februari 2018 bij verweerder een melding gedaan van de overdracht van 305,55 kg (275 

## LLM

In [49]:
!conda install pytorch torchvision cudatoolkit=10.0 -c pytorch

^C


In [46]:
from sentence_transformers import SentenceTransformer

MODEL_STR = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_STR)

ModuleNotFoundError: No module named 'sentence_transformers'